In [1]:
import pandas as pd
from myopic_mces import MCES

smiles_all = pd.read_csv('MassSpecGym.tsv', sep='\t').smiles.unique()
smiles_a = smiles_all[:1000]
smiles_b = smiles_all[1000:2000]

In [2]:
# import matplotlib.pyplot  as plt
# import numpy as np
# import time
# from functools import lru_cache
# hit = 0

# @lru_cache
# def sim_actual(a,b):
#     global hit
#     hit += 1
#     return abs(a - b)

# def sim(a,b):
#     return sim_actual(*sorted((a,b)))
# sim_actual.cache_info
# a = np.random.randint(0, 1000, size=(100))
# b = np.random.randint(0, 1000, size=(100))
# a.sort(),b.sort()
# # ans = np.abs(a[:, None] - b[None, :])
# for e, en in zip(a, a[1:]):
#     sim(e, en)

# for e, en in zip(b, b[1:]):
#     sim(e, en)

# ans = np.zeros((100, 100))
# for i in range(100):
#     for j in range(100):
#         ans[i,j] = sim(a[i],b[j])
# plt.imshow(ans)
# print(hit)

In [9]:
from itertools import product
from tqdm.cli import tqdm
from IPython.display import clear_output
from functools import lru_cache
from myopic_mces import construct_graph
import pulp
import networkx as nx

def MCES_ILP(G1, G2, threshold, solver='default', solver_options={}, no_ilp_threshold=False):
    """
     Calculates the exact distance between two molecules using an ILP

     Parameters
     ----------
     G1 : networkx.classes.graph.Graph
         Graph representing the first molecule.
     G2 : networkx.classes.graph.Graph
         Graph representing the second molecule.
     threshold : float
         Threshold for the comparison. Exact distance is only calculated if the distance is lower than the threshold.
     solver: string
         ILP-solver used for solving MCES. Example:CPLEX_CMD
     solver_options: dict
         additional options to pass to solvers. Example: threads=1, msg=False for better multi-threaded performance
     no_ilp_threshold: bool
         if true, always return exact distance even if it is below the threshold (slower)

     Returns:
     -------
     float
         Distance between the molecules
     int
         Type of Distance:
             1 : Exact Distance
             2 : Lower bound (If the exact distance is above the threshold)

    """

    ILP=pulp.LpProblem("MCES", pulp.LpMinimize)

    #Variables for nodepairs
    nodepairs=[]
    for i in G1.nodes:
        for j in G2.nodes:
            if G1.nodes[i]["atom"]==G2.nodes[j]["atom"]:
                nodepairs.append(tuple([i,j]))
    y=pulp.LpVariable.dicts('nodepairs', nodepairs,
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)
    #variables for edgepairs and weight
    edgepairs=[]
    w={}
    for i in G1.edges:
        for j in G2.edges:
            if (G1.nodes[i[0]]["atom"]==G2.nodes[j[0]]["atom"] and G1.nodes[i[1]]["atom"]==G2.nodes[j[1]]["atom"]) or (G1.nodes[i[1]]["atom"]==G2.nodes[j[0]]["atom"] and G1.nodes[i[0]]["atom"]==G2.nodes[j[1]]["atom"]):
                edgepairs.append(tuple([i,j]))
                w[tuple([i,j])]=max(G1[i[0]][i[1]]["weight"],G2[j[0]][j[1]]["weight"])-min(G1[i[0]][i[1]]["weight"],G2[j[0]][j[1]]["weight"])

    #variables for not mapping an edge
    for i in G1.edges:
        edgepairs.append(tuple([i,-1]))
        w[tuple([i,-1])]=G1[i[0]][i[1]]["weight"]
    for j in G2.edges:
        edgepairs.append(tuple([-1,j]))
        w[tuple([-1,j])]=G2[j[0]][j[1]]["weight"]
    c=pulp.LpVariable.dicts('edgepairs', edgepairs,
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)


    #objective function
    ILP += pulp.lpSum([ w[i]*c[i] for i in edgepairs])




    #Every node in G1 can only be mapped to at most one in G2
    for i in G1.nodes:
        h=[]
        for j in G2.nodes:
            if G1.nodes[i]["atom"]==G2.nodes[j]["atom"]:
                h.append(tuple([i,j]))
        ILP+=pulp.lpSum([y[k] for k in h])<=1

    #Every node in G1 can only be mapped to at most one in G1
    for i in G2.nodes:
        h=[]
        for j in G1.nodes:
            if G1.nodes[j]["atom"]==G2.nodes[i]["atom"]:
                h.append(tuple([j,i]))
        ILP+=pulp.lpSum([y[k] for k in h])<=1

    #Every edge in G1 has to be mapped to an edge in G2 or the variable for not mapping has to be 1
    for i in G1.edges:
        ls=[]
        rs=[]
        for j in G2.edges:
            if (G1.nodes[i[0]]["atom"]==G2.nodes[j[0]]["atom"] and G1.nodes[i[1]]["atom"]==G2.nodes[j[1]]["atom"]) or (G1.nodes[i[1]]["atom"]==G2.nodes[j[0]]["atom"] and G1.nodes[i[0]]["atom"]==G2.nodes[j[1]]["atom"]):
                ls.append(tuple([i,j]))
        ILP+=pulp.lpSum([c[k] for k in ls])+c[tuple([i,-1])]==1

    #Every edge in G2 has to be mapped to an edge in G1 or the variable for not mapping has to be 1
    for i in G2.edges:
        ls=[]
        rs=[]
        for j in G1.edges:
            if (G1.nodes[j[0]]["atom"]==G2.nodes[i[0]]["atom"] and G1.nodes[j[1]]["atom"]==G2.nodes[i[1]]["atom"]) or (G1.nodes[j[1]]["atom"]==G2.nodes[i[0]]["atom"] and G1.nodes[j[0]]["atom"]==G2.nodes[i[1]]["atom"]):
                ls.append(tuple([j,i]))
        ILP+=pulp.lpSum([c[k] for k in ls])+c[tuple([-1,i])]==1

    #The mapping of the edges has to match the mapping of the nodes
    for i in G1.nodes:
        for j in G2.edges:
            ls=[]
            for k in G1.neighbors(i):
                if tuple([tuple([i,k]),j]) in c:
                    ls.append(tuple([tuple([i,k]),j]))
                else:
                    if  tuple([tuple([k,i]),j]) in c:
                        ls.append(tuple([tuple([k,i]),j]))
            rs=[]
            if G1.nodes[i]["atom"]==G2.nodes[j[0]]["atom"]:
                rs.append(tuple([i,j[0]]))
            if G1.nodes[i]["atom"]==G2.nodes[j[1]]["atom"]:
                rs.append(tuple([i,j[1]]))
            ILP+=pulp.lpSum([c[k] for k in ls])<=pulp.lpSum([y[k] for k in rs])


    for i in G2.nodes:
        for j in G1.edges:
            ls=[]
            for k in G2.neighbors(i):
                if tuple([j,tuple([i,k])]) in c:
                    ls.append(tuple([j,tuple([i,k])]))
                else:
                    if tuple([j,tuple([k,i])]) in c:
                        ls.append(tuple([j,tuple([k,i])]))
            rs=[]
            if G2.nodes[i]["atom"]==G1.nodes[j[0]]["atom"]:
                rs.append(tuple([j[0],i]))
            if G2.nodes[i]["atom"]==G1.nodes[j[1]]["atom"]:
                rs.append(tuple([j[1],i]))
            ILP+=pulp.lpSum([c[k] for k in ls])<=pulp.lpSum(y[k] for k in rs)

    #constraint for the threshold
    if threshold!=-1 and not no_ilp_threshold:
        ILP +=pulp.lpSum([ w[i]*c[i] for i in edgepairs])<=threshold
    print(ILP)
    #solve the ILP
    if solver=="default":
        ILP.solve()
    else:
        sol=pulp.getSolver(solver, **solver_options)
        ILP.solve(sol)
    if ILP.status==1:
        return float(ILP.objective.value()),1, ILP
    else:
        return threshold,2,ILP


a,b = construct_graph(smiles_a[0]), construct_graph(smiles_b[0])
d, flag, prob = MCES_ILP(a,b, threshold=10)

MCES:
MINIMIZE
0.5*edgepairs_((0,_1),_(18,_19)) + 0.5*edgepairs_((0,_1),_(18,_23)) + 0.5*edgepairs_((0,_1),_(19,_20)) + 0.5*edgepairs_((0,_1),_(20,_21)) + 0.5*edgepairs_((0,_1),_(21,_22)) + 0.5*edgepairs_((0,_1),_(22,_23)) + 0.5*edgepairs_((0,_1),_(3,_4)) + 0.5*edgepairs_((0,_1),_(3,_8)) + 0.5*edgepairs_((0,_1),_(4,_5)) + 0.5*edgepairs_((0,_1),_(5,_6)) + 0.5*edgepairs_((0,_1),_(6,_7)) + 0.5*edgepairs_((0,_1),_(7,_8)) + 1.0*edgepairs_((0,_1),__1) + 1.0*edgepairs_((1,_2),_(1,_2)) + 1.0*edgepairs_((1,_2),_(2,_3)) + 1.0*edgepairs_((1,_2),_(30,_32)) + 2.0*edgepairs_((1,_2),__1) + 1.0*edgepairs_((1,_3),__1) + 0.5*edgepairs_((10,_11),_(0,_1)) + 0.5*edgepairs_((10,_11),_(13,_14)) + 0.5*edgepairs_((10,_11),_(13,_18)) + 0.5*edgepairs_((10,_11),_(14,_15)) + 0.5*edgepairs_((10,_11),_(15,_16)) + 0.5*edgepairs_((10,_11),_(15,_17)) + 0.5*edgepairs_((10,_11),_(25,_26)) + 0.5*edgepairs_((10,_11),_(26,_27)) + 0.5*edgepairs_((10,_11),_(27,_28)) + 0.5*edgepairs_((10,_11),_(28,_29)) + 0.5*edgepairs_((10,_1

In [16]:
len(x['coefficients'])

KeyError: 'coefficients'

In [22]:
prob.solve?

Signature: prob.solve(solver=None, **kwargs)
Docstring:
Solve the given Lp problem.

This function changes the problem to make it suitable for solving
then calls the solver.actualSolve() method to find the solution

:param solver:  Optional: the specific solver to be used, defaults to the
      default solver.

Side Effects:
    - The attributes of the problem object are changed in
      :meth:`~pulp.solver.LpSolver.actualSolve()` to reflect the Lp solution
File:      ~/micromamba/envs/pb/lib/python3.10/site-packages/pulp/pulp.py
Type:      method

In [ ]:
from itertools import product
from tqdm.cli import tqdm
from IPython.display import clear_output
from functools import lru_cache

data = dict()

def mces_cached(a, b):
    global data
    if (a,b) not in data:
        data[a,b] = MCES(a, b)
    return data[a,b]
def get_bound(a,b):
    assert (a,b) not in data
    data[a,b]
    
def mces(a,b):
    a,b = sorted((a,b))
    return mces_cached(a,b)

for a, b in tqdm(zip(smiles_a[1:], smiles_a)):
    mces(a,b)
    clear_output(wait=True)

for a, b in tqdm(zip(smiles_b[1:], smiles_b)):
    mces(a,b)
    clear_output(wait=True)

for a,b in tqdm(product(smiles_a, smiles_b), total=len(smiles_a)*len(smiles_b), smoothing=0):
    upper_bound = get_bound(a,b) # ac + cb
    

    mces(a, b)
    clear_output(wait=True)

  0%|          | 1307/998686404 [00:15<3279:31:15, 84.59it/s]


KeyboardInterrupt: 

In [20]:
index, distance, time, dist_type = MCES(a, b)
index, distance, time, dist_type

(0, 18.5, 0.010500907897949219, 4)